In [574]:
import random
import networkx as nx
import math

In [575]:
try:
    gph = nx.read_gexf("./contribConnected.gexf")
    print("graph file found at " + str())
    print(gph)

except Exception as e: # if any error
    print(e)
    gph = nx.Graph()

graph file found at 
Graph with 11406 nodes and 286773 edges


In [576]:
values = {}
contributions = set() # all nodes with type contribution
starred = set() # all nodes with type starred repo


In [577]:


def algo(gph):
    values = {}
    contributions = set() # all nodes with type contribution
    starred = set() # all nodes with type starred repo

    for n in gph.nodes(data = True):
        if n[1]['bipartite'] == 0:
            values[n[0]] = 1
        else:
            values[n[0]] = 2

    for n in gph.edges(data = True):
        if 'contributions' in n[2]:
            contributions.add(n[0])
            contributions.add(n[1])
        else:
            starred.add(n[0])
            starred.add(n[1])

    # users -> 1
    # issues -> 2
    # contribution -> 3
    # starred -> 4

    threshold = 3
    predicted = []

    for node in gph.nodes():

        if values[node] == 1: # if it is user
            current_user = node
            weight = {} # to store weights of issues
            prediction = []
            for nodes in gph.neighbors(current_user):
                if values[nodes] == 2: # issues connected to users
                    issues = nodes
                    for issue_neigh in gph.neighbors(issues): # neighbours of issues connected to users
                        if values[issue_neigh] == 2:
                            if issue_neigh in weight:
                                weight[issue_neigh] += 1
                            else:
                                weight[issue_neigh] = 1
                else: # users connected to users
                    user = nodes
                    if user in contributions: # if user is connected via contribution
                        for issue_neigh in gph.neighbors(user): # issues that are neighbours of users connected to cuurent user 
                            if values[issue_neigh] == 2:
                                if issue_neigh in weight:
                                    weight[issue_neigh] += 2
                                else:
                                    weight[issue_neigh] = 2
                    else: # if user is connected via starred repo
                        for issue_neigh in gph.neighbors(user): # issues that are neighbours of users connected to cuurent user
                            if values[issue_neigh] == 2:
                                if issue_neigh in weight:
                                    weight[issue_neigh] += 1
                                else:
                                    weight[issue_neigh] = 1
            
            Issues = list(weight.keys())
            Weights = list(weight.values())

            # print(current_user, end = " --> ")
            # Probability = Element Weight / Sum Of All Weights
            # predicted_issue = random.choices(Issues, Weights, k = 1)[0]
            # print(predicted_issue) # k is the number of outputs
            sorted_prediction = sorted(weight.items(), key=lambda x: x[1], reverse=True)
            # print(sorted_prediction)
            for x in range(0,len(sorted_prediction)):
                if(sorted_prediction[x][1] > 1):
                    predicted_issue = sorted_prediction[x][0]
                    predicted.append([current_user, predicted_issue])
    return predicted



In [578]:
B= gph


users = [n for n, d in B.nodes(data=True) if d["bipartite"] == 0]
issues = [n for n, d in B.nodes(data=True) if d["bipartite"] == 1]

n = len(issues)

In [579]:
def removeEdges(graph, n):
    g = nx.Graph()
    initial = graph.number_of_edges()
    issuesList = random.sample(issues, n)
    for i in issuesList:
        for j in list(graph.neighbors(i)):
            if graph.nodes[j]["bipartite"] == 0:
                g.add_node(i, bipartite=1)
                g.add_node(j, bipartite=0)
                g.add_edge(i,j)
                graph.remove_edge(i,j)
    final = graph.number_of_edges()
    print(initial - final)
    return g,graph

B = gph.copy()
train, test = removeEdges(B,2000)

2000


In [580]:
predicted = algo(test)

In [581]:
og_list = train.edges()
og_list = list(og_list)

# print(og_list)
common_edges = []

for edge1 in predicted:
    for edge2 in og_list:
        # print("-------------")
        
        sorted_edge1 = tuple(sorted(edge1))
        sorted_edge2 = tuple(sorted(edge2))
        # print(sorted_edge1)
        # print(sorted_edge2)
        if sorted_edge1 == sorted_edge2:
            # print(edge1)
            # print(edge2)
            # print(str(sorted_edge1) + "--------" + str(sorted_edge2))
            common_edges.append(edge1)
            break

print(len(common_edges))
for i in common_edges:
    print(i)


8
['lapce', 'PR_kwDOBy2NM85LQJYb']
['login', 'MDExOlB1bGxSZXF1ZXN0NjIyMTY2OTU=']
['login', 'MDExOlB1bGxSZXF1ZXN0NTkzMDE2MTQ=']
['kaina404', 'MDExOlB1bGxSZXF1ZXN0Mjc4NTA4Mjc0']
['kaina404', 'MDExOlB1bGxSZXF1ZXN0Mjc2NTQyNjA4']
['SandPoot', 'PR_kwDOA7lyys5KeDbQ']
['SS13-SCP13', 'MDExOlB1bGxSZXF1ZXN0MjQxMDA4NTQz']
['FarmBot', 'PR_kwDOAPHTvM5LJ4Ko']
